In [1]:
from dotenv import load_dotenv
# import time

load_dotenv()


True

In [2]:
question = '신생아 가구 뭐지?'

In [3]:
# 임베딩
from openai import OpenAI
client = OpenAI()


def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [4]:
# db 검색
# annc_id = 1

question_emb = get_embedding(question)


In [5]:
type(question_emb)

list

In [6]:
from database import DataBaseHandler


with DataBaseHandler() as db:
    query = """
    SELECT
        annc_id,
        chunk_id,
        CHUNK_TEXT,
        chunk_type,
        EMBEDDING <=> %s::vector AS distance,  -- '<=>' 연산자는 코사인 거리를 계산
        'VECTOR' AS SEARCH_TYPE
    FROM
        DOC_CHUNKS
    WHERE
        annc_id = 350
        and chunk_type not in ('heading')
    UNION ALL
    SELECT
        annc_id,
        chunk_id,
        CHUNK_TEXT,
        chunk_type,
        0.5 AS distance,
        'TABLE' AS SEARCH_TYPE
    FROM
        DOC_CHUNKS
    WHERE
        annc_id = 350
        and chunk_type not in ('heading')
        and chunk_text like %s
    ORDER BY
        distance
    LIMIT 5
    """
    result = db.execute_query(query, (question_emb,f"%{question}%"))


In [7]:
result

[{'annc_id': 350,
  'chunk_id': 64146,
  'chunk_text': '■ 임대기간 : 2년\n- 최초 임대기간 경과 후 재계약 기준 충족 시 2년 단위로 9회 재계약 가능(최장 20년 거주 가능)\n- 입주 후 다자녀가구(두 명 이상의 직계비속을 양육하는 가구)가 된 경우, 유형 전환 후 2년 단위로 9회 재계약 가능\n■ 재계약기준\n- 재계약 시점에 재계약 기준(무주택세대구성원, 전년도 도시근로자 가구원수별 가구당 월평균소득의 105%이하, 국민임대주택 자산기준)을 충족하여야 하며, 소득 또는 자산이 재계약 기준을 초과한 경우에도 1회에 한해 재계약이 가능하며, 이 경우 임대료 등을 80% 할증하여 적용\n- 재계약 시점에 입주당시 입주자격 소득요건 초과 시 아래 할증구간별 할증비율 적용하여 임대료 등 할증',
  'chunk_type': 'text',
  'distance': 0.7464845029404459,
  'search_type': 'VECTOR'},
 {'annc_id': 350,
  'chunk_id': 64154,
  'chunk_text': '소득 · 자산 기준 및 자격 검증 대상(이후 ‘세대구성원’을 말함)\n| 구분 | 2인가구 | 3인가구 | 4인가구 | 5인가구 | 6인가구 | 7인가구 | 8인가구 |\n| --- | --------- | --------- | --------- | --------- | --------- | --------- | ---------- |\n| 70% | 4,381,602 | 5,338,881 | 6,004,662 | 6,321,734 | 6,813,160 | 7,304,587 | 7,796,013 |\n| 90% | 5,477,003 | 6,864,276 | 7,720,279 | 8,127,943 | 8,759,777 | 9,391,612 | 10,023,446 |\n| 구분 | 자녀가 없거나, 2023.3.27. 이전 출생한 자녀만 있는 경우 | 2023.3.28. 이후

In [8]:
# import psycopg
# import numpy as np
# import json
# from typing import List

# # ----------------------------------------
# # 1. DB 연결 설정 (사용자 환경에 맞게 변경하세요)
# # ----------------------------------------
# DB_CONN_INFO = "host=localhost dbname=your_db user=your_user password=your_password"

# # ----------------------------------------
# # 2. 쿼리 벡터(임베딩) 생성
# #    (실제로는 임베딩 모델을 호출하여 생성합니다)
# # ----------------------------------------
# # 1536차원의 랜덤 벡터 생성 예시 (실제 임베딩이라 가정)
# QUERY_DIMENSION = 1536
# query_vector_np = np.random.rand(QUERY_DIMENSION).astype(np.float32)

# # ----------------------------------------
# # 3. pgvector 형식의 문자열로 변환하는 함수
# # ----------------------------------------
# def vector_to_pg_string(vector: np.ndarray) -> str:
#     """
#     NumPy 배열을 pgvector가 인식하는 문자열 ([a, b, c, ...])로 변환합니다.
#     """
#     # 소수점 6자리까지로 포매팅하여 정밀도를 유지
#     formatted_vector = [f"{x:.6f}" for x in vector]
#     # PostgreSQL의 VECTOR 타입 문자열 형식으로 변환하여 반환
#     return f"[{', '.join(formatted_vector)}]"


# # ----------------------------------------
# # 4. SQL 쿼리 실행
# # ----------------------------------------
# def search_vector_db(query_vector: np.ndarray, limit: int = 5):
#     # 쿼리 벡터를 PostgreSQL이 처리할 수 있는 문자열로 변환
#     pg_vector_string = vector_to_pg_string(query_vector)

#     # SQL 쿼리 정의:
#     # '%s' 플레이스홀더를 사용하여 쿼리 문자열을 안전하게 바인딩합니다.
#     # '<=>' 연산자는 pgvector의 코사인 거리(Cosine Distance) 연산자입니다.
#     sql_query = """
#     SELECT
#         chunk_text,
#         embedding <=> %s AS distance,
#         file_id
#     FROM
#         doc_chunks
#     ORDER BY
#         distance
#     LIMIT %s;
#     """

#     try:
#         # DB 연결 및 커서 생성
#         with psycopg.connect(DB_CONN_INFO) as conn:
#             with conn.cursor() as cur:
#                 # 쿼리 실행 시 pg_vector_string와 limit 값을 안전하게 바인딩
#                 cur.execute(sql_query, (pg_vector_string, limit))
                
#                 # 결과 가져오기
#                 results = cur.fetchall()

#                 # 컬럼 이름과 함께 결과 출력
#                 print(f"--- Top {limit} 검색 결과 (distance가 0에 가까울수록 유사) ---")
#                 for row in results:
#                     chunk_text, distance, file_id = row
#                     print(f"File ID: {file_id}, Distance: {distance:.4f}")
#                     # 긴 텍스트는 보기 좋게 잘라서 출력
#                     print(f"  Chunk Text: {chunk_text[:80]}...")
                
#     except psycopg.Error as e:
#         print(f"데이터베이스 오류 발생: {e}")
        
# # 함수 실행
# search_vector_db(query_vector_np)